In [122]:
import pandas as pd
import psycopg2

In [123]:
df_crash=pd.read_csv('/Users/joe/Desktop/ETL Pipelines/data/transformation_data/traffic_crashes.csv')
df_vehicle_crash=pd.read_csv('/Users/joe/Desktop/ETL Pipelines/data/transformation_data/traffic_crash_vehicle.csv')
df_crash['crash_date']=pd.to_datetime(df_crash['crash_date'])
df_vehicle_crash['crash_date']=pd.to_datetime(df_crash['crash_date'])


In [124]:
keep_1=['crash_record_id','crash_date','posted_speed_limit',	'traffic_control_device','weather_condition','lighting_condition','crash_hour','crash_day_of_week','crash_month','latitude','longitude']
df_new_crashes=df_crash[keep_1]



In [125]:
df_vehicle_crash.dropna(axis=1,thresh=200,inplace=True)
keep_2=['crash_unit_id','crash_record_id','rd_no', 'crash_date','unit_no','unit_type','vehicle_id' ,'make','model','lic_plate_state' ,'vehicle_defect','maneuver','occupant_cnt']
df_new_vehicle_crashes=df_vehicle_crash[keep_2]
df_new_vehicle_crashes

,crash_unit_id,crash_record_id,rd_no,crash_date,unit_no,unit_type,vehicle_id,make,model,lic_plate_state,vehicle_defect,maneuver,occupant_cnt
0,1255349,305b06235b250aa0029c07313c84f969f4bc13c1cc3715...,JE494008,2021-12-31 14:00:00,1,DRIVER,1191605.0,DODGE,CHARGER,IL,UNKNOWN,TURNING LEFT,1.0
1,1255350,305b06235b250aa0029c07313c84f969f4bc13c1cc3715...,JE494008,2021-12-31 14:00:00,2,DRIVER,1191608.0,KIA,SEDONA,IL,UNKNOWN,STRAIGHT AHEAD,1.0
2,1255351,305b06235b250aa0029c07313c84f969f4bc13c1cc3715...,JE494008,2021-12-31 13:56:00,3,PARKED,1191614.0,CHRYSLER,PACIFICA,IL,NONE,PARKED,0.0
3,1255367,530411c8611eb0ccb9b25f16b2955cd21761fa1928dcaa...,JE494048,2021-12-31 13:46:00,1,DRIVER,1191620.0,TOYOTA,TUNDRA,IL,NONE,U-TURN,1.0
4,1255368,530411c8611eb0ccb9b25f16b2955cd21761fa1928dcaa...,JE494048,2021-12-31 13:45:00,2,DRIVER,1191622.0,GENERAL MOTORS CORPORATION (GMC),ENVOY,IL,NONE,STRAIGHT AHEAD,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1254361,a6b386bec506fa8f21dde8b20f9899ccd737969864df7c...,JE491940,2021-12-27 03:00:00,1,DRIVER,1190674.0,UNKNOWN,OTHER (EXPLAIN IN NARRATIVE),NaN,UNKNOWN,UNKNOWN/NA,1.0
996,1254362,a6b386bec506fa8f21dde8b20f9899ccd737969864df7c...,JE491940,2021-12-27 02:43:00,2,PARKED,1190680.0,TOYOTA,CAMRY,IL,UNKNOWN,PARKED,0.0
997,1254209,5024bcbf415446252aa87dda2c035718a3a28916ed8e00...,JE491697,2021-12-27 02:35:00,1,DRIVER,1190540.0,"PETERBILT MOTORS COMPANY (DIVISION OF PACCAR, ...","PETERBILT MOTORS CO., DIVISION PACCAR, INC.",IL,NONE,BACKING,1.0
998,1254210,5024bcbf415446252aa87dda2c035718a3a28916ed8e00...,JE491697,2021-12-27 02:05:00,2,DRIVER,1190550.0,BUICK,OTHER (EXPLAIN IN NARRATIVE),IL,NONE,STRAIGHT AHEAD,1.0


In [126]:
#connecting to postgres
conn = psycopg2.connect(
    database="chicago_crash_data",
    user="postgres",
    password="3884",
    host="localhost", 
    port="5432")

#establishing a cursor for execution of queries
cur = conn.cursor()

In [127]:
#Query vehicle
insert_query_vehicle = '''INSERT INTO
      chicago_dmv.Vehicle(crash_unit_id,crash_record_id,rd_no,crash_date, unit_no,unit_type,vehicle_id,make,model,lic_plate_state,vehicle_defect,maneuver,occupant_cnt)VALUES
      (%s, %s, %s, %s, %s, %s, %s, %s,%s, %s, %s, %s, %s) ON CONFLICT (crash_unit_id) DO NOTHING;
      '''

insert_query_crash= '''INSERT INTO
      chicago_dmv.Crash(crash_record_id ,crash_date,posted_speed_limit,traffic_control_device ,weather_condition,lighting_condition,crash_hour,crash_day_of_week ,crash_month ,latitude ,longitude )VALUES
      (%s, %s, %s, %s, %s, %s, %s, %s,%s, %s, %s)ON CONFLICT (crash_record_id) DO NOTHING;
      '''

In [128]:
#loading data to postgres
for index , row in df_new_vehicle_crashes.iterrows():
    values_vehicle = (
            row['crash_unit_id'],
            row['crash_record_id'],
            row['rd_no'],
            row['crash_date'],
            row['unit_no'],
            row['unit_type'],
            row['vehicle_id'],
            row['make'],
            row['model'],
            row['lic_plate_state'],
            row['vehicle_defect'],
            row['maneuver'],
            row['occupant_cnt']
    )
    cur.execute(insert_query_vehicle, values_vehicle)                         

In [129]:
for index, row in df_new_crashes.iterrows():
    values_crash = (
            row['crash_record_id'],
            row['crash_date'],
            row['posted_speed_limit'],
            row['traffic_control_device'],
            row['weather_condition'],
            row['lighting_condition'],
            row['crash_hour'],
            row['crash_day_of_week'],
            row['crash_month'],
            row['latitude'],
            row['longitude']
    )
    cur.execute(insert_query_crash, values_crash)

In [130]:
#commiting changes to database
conn.commit()


In [131]:
# Close the cursor and database connection
cur.close()
conn.close()